### 规则法
请教了一个虚伪的大佬学姐，才知道时间序列还有一种规则法，给了dropout大佬的简书文章，在这里试一试。

* [时间序列规则法快速入门](https://www.jianshu.com/p/31e20f00c26f)

#### 调试中位数和平均数

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as mse

In [ ]:
all_feature = np.zeros((81, 2, 24, 144))
for istation in range(81):
    station_file = pd.read_csv('../Metro_station/station-'+str(istation).zfill(2)+'.csv')
    for jinout in range(2):
        for kday in range(24):
            if jinout == 0:
                station_feature = station_file.iloc[144*kday:144*(kday+1), 2].values
            else:
                station_feature = station_file.iloc[144*kday:144*(kday+1), 6].values
            all_feature[istation, jinout, kday, :] = station_feature

In [ ]:
train_feature = all_feature[:, :, :23, :]
test_feature = all_feature[:, :, [-1], : ]

In [ ]:
station_feature = train_feature[istation, jinout]
station_mean = np.mean(station_feature, axis=1)
station_mean = np.expand_dims(station_mean, axis=1)
station_feature_normal = station_feature/station_mean
station_feature_median = np.median(station_feature_normal, axis=0)
station_feature_mean = np.mean(station_feature_normal, axis=0)
station_feature_mix = (station_feature_median + station_feature_mean)/2
mse(station_feature_mix*station_mean[-1][-1],test_feature[istation, jinout, 0, :])

In [ ]:
loss = 0
for istation in range(81):
    if istation == 54:
        continue
    for jinout in range(2):
        station_feature = train_feature[istation, jinout]
        station_mean = np.mean(station_feature, axis=1)
        station_mean = np.expand_dims(station_mean, axis=1)
        station_feature_normal = station_feature/station_mean
        station_feature_median = np.median(station_feature_normal, axis=0)
        station_feature_mean = np.mean(station_feature_normal, axis=0)
        station_feature_mix = (station_feature_median*4 + station_feature_mean*6)/10
        loss += mse(np.ceil(station_feature_mix*station_mean[-1][-1]),test_feature[istation, jinout, 0, :])
print(loss)

#### 聚合28日数据

In [ ]:
metro_file = pd.read_csv('../Metro_testA/testA_record_2019-01-'+str(pred_day).zfill(2)+'.csv')
metro_file.time = pd.to_datetime(metro_file.time)
start_time = '2019-01-'+str(pred_day)+' 00:00:00'
start_time = pd.to_datetime(start_time)
end_time = '2019-01-'+str(pred_day)+' 00:10:00'
end_time = pd.to_datetime(end_time)
tenmin_delta = end_time - start_time
metro_dfs = []
for k in range(81):
    metro_dfs.append(pd.DataFrame(np.zeros((144, 11))))
for j in range(144):
    metro_tmp = metro_file[(metro_file.time>=start_time) & (metro_file.time<end_time)].groupby(['stationID', 'status', 'payType']).count()
    for k in range(81):
        try:
            metro_dfs[k].loc[j, 0] = metro_tmp.loc[k, 1].time.sum()
        except:
            metro_dfs[k].loc[j, 0] = 0
        try:
            metro_dfs[k].loc[j, 1] = metro_tmp.loc[k, 1, 0].time
        except:
            metro_dfs[k].loc[j, 1] = 0
        try:
            metro_dfs[k].loc[j, 2] = metro_tmp.loc[k, 1, 1].time
        except:
            metro_dfs[k].loc[j, 2] = 0
        try:
            metro_dfs[k].loc[j, 3] = metro_tmp.loc[k, 1, 2].time
        except:
            metro_dfs[k].loc[j, 3] = 0
        try:
            metro_dfs[k].loc[j, 4] = metro_tmp.loc[k, 1, 3].time
        except:
            metro_dfs[k].loc[j, 4] = 0
        try:
            metro_dfs[k].loc[j, 5] = metro_tmp.loc[k, 0].time.sum()
        except:
            metro_dfs[k].loc[j, 5] = 0
        try:
            metro_dfs[k].loc[j, 6] = metro_tmp.loc[k, 0, 0].time
        except:
            metro_dfs[k].loc[j, 6] = 0
        try:
            metro_dfs[k].loc[j, 7] = metro_tmp.loc[k, 0, 1].time
        except:
            metro_dfs[k].loc[j, 7] = 0
        try:
            metro_dfs[k].loc[j, 8] = metro_tmp.loc[k, 0, 2].time
        except:
            metro_dfs[k].loc[j, 8] = 0
        try:
            metro_dfs[k].loc[j, 9] = metro_tmp.loc[k, 0, 3].time
        except:
            metro_dfs[k].loc[j, 9] = 0
        if start_time.weekday()<=4:
            metro_dfs[k].loc[j, 10] = 0
        else:
            metro_dfs[k].loc[j, 10] = 1
    start_time = end_time
    end_time += tenmin_delta

In [ ]:
for i in range(81):
    metro_dfs[i].rename(columns={0:'in_num', 1:'in_paytype_0', 2:'in_paytype_1', 3:'in_paytype_2', 4:'in_paytype_3', 
                                 5:'out_num', 6:'out_paytype_0', 7:'out_paytype_1', 8:'out_paytype_2', 9:'out_paytype_3',
                                 10:'weekend'}, inplace=True)

In [ ]:
eval_features = np.zeros((81, 2, 1, 144))
for istation in range(81):
    station_feature = metro_dfs[istation]
    for jinout in range(2):
        if jinout == 0:
            eval_features[istation, jinout, 0, :] = station_feature.iloc[:,0]
        else:
            eval_features[istation, jinout, 0, :] = station_feature.iloc[:,5]

In [ ]:
all_feature = np.concatenate((all_feature, eval_features), axis=2)

#### 生成输出

In [ ]:
result = np.zeros((81*144, 2))
for istation in range(81):
    if istation == 54:
        continue
    for jinout in range(2):
        station_feature = all_feature[istation, jinout]
        station_mean = np.mean(station_feature, axis=1)
        station_mean = np.expand_dims(station_mean, axis=1)
        station_feature_normal = station_feature/station_mean
        station_feature_median = np.median(station_feature_normal, axis=0)
        station_feature_mean = np.mean(station_feature_normal, axis=0)
        station_feature_mix = (station_feature_median*4 + station_feature_mean*6)/10
        if jinout == 0:
            result[istation*144:(istation+1)*144, 0] = np.ceil(station_feature_mix*station_mean[-1][-1])
        else:
            result[istation*144:(istation+1)*144, 1] = np.ceil(station_feature_mix*station_mean[-1][-1])

In [ ]:
result = pd.DataFrame(result)
result.to_csv('pred_28.csv', index=False)